In [29]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [30]:
from sklearn.model_selection import train_test_split
# Загрузка данных с первого листа файла Excel
df1 = pd.read_excel('../data/dataSet.xlsx', sheet_name='Программы')
df2 = pd.read_excel('../data/dataSet.xlsx', sheet_name='Предприятия')


In [31]:
# Предварительная обработка текста
df1['conditions'] = df1['conditions'].str.lower().str.replace('[^\w\s]', '')
df2['description'] = df2['description'].str.lower().str.replace('[^\w\s]', '')


In [39]:
# Преобразование строки в список
df2['programms'] = df2['programms'].str.split(',')

# "Взрыв" списка
df2 = df2.explode('programms')

# Преобразование типа данных для корректного слияния
df2['programms'] = df2['programms'].astype(int)
df1['number'] = df1['number'].astype(int)

# Объединение двух DataFrame
df = pd.merge(df1, df2, left_on='number', right_on='programms')

# Удаление ненужного столбца
df = df.drop(columns=['programms'])
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df

AttributeError: Can only use .str accessor with string values!

In [33]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from rake_nltk import Rake
import nltk

In [34]:
# Устанавливаем стоп-слова для русского языка
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\artem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
# Объединение текста описания предприятия и текста критериев участия
df['combined_text'] = df['description'] + ' ' + df['conditions']

In [36]:
# Извлечение ключевых слов с помощью TF-IDF
tfidf = TfidfVectorizer(max_features=1000, stop_words=list(stop_words))
X_tfidf = tfidf.fit_transform(df['combined_text'])
y = df['number']

In [37]:
# Создание обучающей и тестовой выборки
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.8, random_state=42)

In [38]:
# Обучение модели на основе TF-IDF
rf_tfidf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_tfidf.fit(X_train_tfidf, y_train)

# Предсказание на тестовой выборке и оценка точности
y_pred_tfidf = rf_tfidf.predict(X_test_tfidf)
accuracy_tfidf = accuracy_score(y_test, y_pred_tfidf)
print("Accuracy using TF-IDF:", accuracy_tfidf)

Accuracy using TF-IDF: 1.0
